In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.shape

(918, 12)

In [4]:
mask = df.isnull()
total = mask.sum()
percent = 100*mask.mean()

missing_data = pd.concat([total, percent], axis=1,join='outer',
                         keys=['count_missing', 'perc_missing'])
missing_data.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data

,count_missing,perc_missing
Age,0,0.0
Sex,0,0.0
ChestPainType,0,0.0
RestingBP,0,0.0
Cholesterol,0,0.0
FastingBS,0,0.0
RestingECG,0,0.0
MaxHR,0,0.0
ExerciseAngina,0,0.0
Oldpeak,0,0.0


In [5]:
x = df.drop(['HeartDisease', 'Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope' ], axis = 'columns')
y = df.HeartDisease

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-1.4331398 ,  0.41090889,  0.82507026, -0.55134134,  1.38292822,
        -0.83243239],
       [-0.47848359,  1.49175234, -0.17196105, -0.55134134,  0.75415714,
         0.10566353],
       [-1.75135854, -0.12951283,  0.7701878 , -0.55134134, -1.52513802,
        -0.83243239],
       ...,
       [ 0.37009972, -0.12951283, -0.62016778, -0.55134134, -0.85706875,
         0.29328271],
       [ 0.37009972, -0.12951283,  0.34027522, -0.55134134,  1.4615246 ,
        -0.83243239],
       [-1.64528563,  0.30282455, -0.21769643, -0.55134134,  1.42222641,
        -0.83243239]])

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=30)

print("input - training set :", x_train.shape)
print("output - training set :", y_train.shape)
print("input - validation set :", x_test.shape)
print("output - validation set :", y_test.shape)

input - training set : (734, 6)
output - training set : (734,)
input - validation set : (184, 6)
output - validation set : (184,)


In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.7554347826086957

# Use PCA to reduce dimensions

In [10]:
x

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak
0,40,140,289,0,172,0.0
1,49,160,180,0,156,1.0
2,37,130,283,0,98,0.0
3,48,138,214,0,108,1.5
4,54,150,195,0,122,0.0
...,...,...,...,...,...,...
913,45,110,264,0,132,1.2
914,68,144,193,1,141,3.4
915,57,130,131,0,115,1.2
916,57,130,236,0,174,0.0


# Use components such that 95% of variance is retained

In [11]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
x_pca = pca.fit_transform(x)
x_pca.shape

(918, 2)

In [12]:
x_pca

array([[ 92.31059589, -29.43596873],
       [-17.1439375 , -13.737187  ],
       [ 81.90634696,  38.23663195],
       ...,
       [-69.00315763,  17.32523762],
       [ 39.20577805, -33.59550121],
       [-21.43780112, -37.20765194]])

# Logistic Regression

In [25]:
x_train_pca, x_test_pca, y_train, y_test = train_test_split(x_pca, y, test_size=0.3, random_state=30)

In [26]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(x_train_pca, y_train)
model.score(x_test_pca, y_test)

0.7065217391304348

# RandomForestClassifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_train_pca, y_train)
model_rf.score(x_test_pca, y_test)

0.677536231884058

# SVM

In [28]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train_pca, y_train)
svc.score(x_test_pca, y_test)

0.7427536231884058